In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import warnings ; warnings.filterwarnings(action='ignore')

%pwd

'/Users/yejin/Desktop/Career/Projects/Review_Helpfulness/crawling'

In [31]:
# URL = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008'
driver = webdriver.Chrome(executable_path='/Users/yejin/Downloads/chromedriver')
driver.get(url=URL)

In [3]:
url_list = []

# 첫 화면 url 크롤링
urls=driver.find_elements_by_css_selector(".prd_info>a")

for url in urls:
    url=url.get_attribute('href')
    url_list.append(url)
    
# print(len(url_list))
# url_list

# 2~n번째 화면 url 크롤링
for i in range(1, 10):
    driver.find_element_by_xpath('//*[@id="Container"]/div[2]/a[' + str(i) + ']').click()
    time.sleep(3)
    
    urls=driver.find_elements_by_css_selector(".prd_info>a")
    
    for url in urls:
        url=url.get_attribute('href')
        url_list.append(url)
    

In [4]:
# url 저장
olive_young_skin_url = pd.DataFrame(url_list, columns = ['link'])
olive_young_skin_url.to_csv("url_list_0115.csv", index=False)

In [5]:
len(url_list)

240

In [23]:
DATA = pd.DataFrame(columns = ['name', 'image', 'review', 'vote'])

In [39]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=97.0.4692.71)
Stacktrace:
0   chromedriver                        0x0000000100c54280 chromedriver + 4833920
1   chromedriver                        0x0000000100be8bf8 chromedriver + 4393976
2   chromedriver                        0x00000001007dec84 chromedriver + 158852
3   chromedriver                        0x00000001007c250c chromedriver + 42252
4   chromedriver                        0x000000010082a244 chromedriver + 467524
5   chromedriver                        0x000000010082dde8 chromedriver + 482792
6   chromedriver                        0x00000001008037e0 chromedriver + 309216
7   chromedriver                        0x0000000100c16828 chromedriver + 4581416
8   chromedriver                        0x0000000100c2b450 chromedriver + 4666448
9   chromedriver                        0x0000000100c2fd1c chromedriver + 4685084
10  chromedriver    

In [32]:
from tqdm import tqdm
# 상품 리뷰정보 크롤링
for URL in tqdm(url_list[120:]):
    driver.get(url=URL)
    time.sleep(1)

    # 상품명
    name = driver.find_element_by_xpath('//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
    # name.append(get_name.text)

    # 이미지
    image = driver.find_element_by_css_selector('#mainImg').get_attribute('src')
    # 리뷰 버튼
    review_button = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
    review_button.click()

    time.sleep(2.3)

    for i in range(2, 5):
        # 리뷰 정보
        try:
            review_txts = driver.find_elements_by_css_selector(".txt_inner")
            votes = driver.find_elements_by_css_selector('div.recom_area span.num')
            for k in range(10):
                DATA = DATA.append({'name':name, 'image':image, 'review':review_txts[k].text, 'vote': int(votes[i].text)}, ignore_index=True)
            time.sleep(0.3)
        except:
            time.sleep(1)
            review_txts = driver.find_elements_by_css_selector(".txt_inner")
            votes = driver.find_elements_by_css_selector('div.recom_area span.num')
            for k in range(len(review_txts)):
                DATA = DATA.append({'name':name, 'image':image, 'review':review_txts[k].text, 'vote': int(votes[k].text)}, ignore_index=True)
            time.sleep(0.3)
        
        try:
            driver.find_element_by_css_selector(f'#gdasContentsArea > div > div.pageing > a:nth-child({i})').click()
            time.sleep(1)
        except:
            pass
        time.sleep(1)

100%|██████████| 120/120 [25:53<00:00, 12.95s/it]


In [22]:
review_txts

[]

In [33]:
DATA = DATA.drop_duplicates()

In [34]:
len(DATA.drop_duplicates())

6438

In [35]:
DATA.to_csv("./olive_young_reviews.csv", index = False)

In [38]:
DATA.review

0       >>피부타입:저는 민감성입니닷!! 조금이라도 안맞는걸 바르면 바로 알러지반응이 일어...
1       아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받아서...
2       지난번 올영세일때 구매했는데 너무 좋아서\n이번에 또 세일하길래 쟁여놓으려고 구매했...
3       @포장~선물하기 딱좋은 깔끔 예쁜 박스에 담겨져왔어요 선물받는듯 기분좋은 포장입니다...
4       ⁉️다들 리뷰 너무 좋으신데... 단점리뷰입니다🥺 \n✔️로션이지만 확실한 보습!\...
                              ...                        
6692    피부과에서 여드름 레이저 치료 받을 때 처방 받아서 한동안 사용했던 제품입니다. 요...
6693                    재재재구매템. 진짜 좋은데 양이 너무 적고 비쌈 ㅜ그게 아쉽
6694    4-5년정도 사용중인데, 잠깐 호기심에 다른 브랜드를 사용해보다가도 며칠이 지나 다...
6695                   만족하는데 양이 너어어어무 작음 ㅜ 금방써서 돈이 줄줄셈 ㅜㅜ
6696    더마 브랜드들을 좋아하는데 이 제품은 좀 실망스럽네요\n따가워요... 라로 제품 중...
Name: review, Length: 6438, dtype: object

In [37]:
DATA.vote.unique()

array([5, 4, 1, 25, 38, 12, 3, 0, 2, 7, 65, 66, 46, 32, 6, 10, 29, 194,
       19, 15, 16, 22, 11, 9, 14, 8, 47, 37, 39, 26, 18], dtype=object)